In [3]:
!pip install -q torch peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 accelerate einops tqdm scipy datasets huggingface_hub

import os
from dataclasses import dataclass, field
from typing import Optional
import torch
from datasets import load_dataset, DatasetDict
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
)
from tqdm.notebook import tqdm
from trl import SFTTrainer
from huggingface_hub import notebook_login

# Log in to Hugging Face
notebook_login()





In [6]:
# Load the dataset
dataset = load_dataset("brainchalov/pubmed_arxiv_abstracts_data", split="train[:1%]")

# Convert dataset to DataFrame for easier manipulation
import pandas as pd
df = pd.DataFrame(dataset)
df.head(5)
df.shape
df.info()

Repo card metadata block was not found. Setting CardData to empty.


In [7]:
# Format the dataset
def format_row(row):
    abstr = row["abstr"]
    title = row["title"]
    labels = f"{row['label_journal']} {row['label_field']}"
    formatted_string = f"[TITLE] {title} [ABSTRACT] {abstr} [LABELS] {labels}"
    return formatted_string

df['Formatted'] = df.apply(format_row, axis=1)

# Rename and save the formatted dataset
new_df = df.rename(columns={'Formatted': 'Text'})
new_df = new_df[['Text']]
new_df.to_csv('formatted_dataset.csv', index=False)

# Reload the dataset in CSV format for training
training_dataset = load_dataset("csv", data_files="formatted_dataset.csv", split="train")

# Load the base model and tokenizer
base_model = "microsoft/phi-2"
new_model = "phi-2-text-generation"

tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    trust_remote_code=True,
    flash_attn=True,
    flash_rotary=True,
    fused_dense=True,
    low_cpu_mem_usage=True,
    device_map="auto",  # Automatically shards across available GPUs
    revision="refs/pr/23",
)

# Prepare the model for k-bit training
model.config.use_cache = False
model.config.pretraining_tp = 1
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

# Set up training arguments
training_arguments = TrainingArguments(
    output_dir="./text_generation",
    num_train_epochs=1,
    per_device_train_batch_size=1,  # Reduced batch size to save memory
    gradient_accumulation_steps=32,
    evaluation_strategy="steps",
    eval_steps=1000,
    logging_steps=500,
    optim="paged_adamw_8bit",
    learning_rate=3e-5,
    lr_scheduler_type="cosine",
    save_steps=1000,
    warmup_ratio=0.1,
    weight_decay=0.01,
    max_steps=-1,
    fp16=True,  # Use mixed precision training
)

# Configure PEFT (Parameter-Efficient Fine-Tuning)
peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["Wqkv", "fc1", "fc2"],
)

# Initialize the Trainer and start training
trainer = SFTTrainer(
    model=model,
    train_dataset=training_dataset,
    peft_config=peft_config,
    dataset_text_field="Text",
    max_seq_length=128,  # Reduce max sequence length
    tokenizer=tokenizer,
    args=training_arguments,
)

# Clear CUDA cache to free up memory
torch.cuda.empty_cache()

# Start training
trainer.train()


Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/5003 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:482: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss


TrainOutput(global_step=156, training_loss=2.1830845368214145, metrics={'train_runtime': 1833.961, 'train_samples_per_second': 2.728, 'train_steps_per_second': 0.085, 'total_flos': 5216656990771200.0, 'train_loss': 2.1830845368214145, 'epoch': 1.0})